## KeyBert를 이용해 키워드 추출

# 1단계. 환경설정

In [ ]:
!pip install -U transformers datasets torch pandas sentencepiece accelerate keybert


다운받은 모듈을 import 한다. GPU check

In [ ]:
import torch
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from keybert import KeyBERT

device = 0 if torch.cuda.is_available() else -1
print(f"Using GPU? {torch.cuda.is_available()}  -> device={device}")


# 2단계. 데이터셋 로드 및 준비


In [ ]:
# (단계 2) 데이터셋 로드 및 준비
ds = load_dataset("ajaykarthick/imdb-movie-reviews")
print(ds)
print(ds["train"][0])

# 실습용 소규모 subset
subset = ds["train"].shuffle(seed=42).select(range(200))
subset


# 3단계. 파이프라인 불러오기

In [ ]:
# (2) BERT 기반 키워드 추출기 로드
model_name = "yanekyuk/bert-keyword-extractor"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name).to(device)

kw_model = KeyBERT(model=bert_model)


# 4단계. 모델 적용 및 컬럼 추가

In [ ]:
def extract_keywords_keybert(text, top_k=5):
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1, 2),   # 단일 단어 ~ 2-gram
        stop_words="english",           # 불용어 제거
        top_n=top_k,
        use_maxsum=True                 # 다양성 확보 (옵션)
    )
    # 결과는 [(키워드, score), ...] 형태
    return [kw for kw, score in keywords]

In [ ]:
def add_keywords(batch):
    return {"keywords": [extract_keywords_keybert(r, top_k=3) for r in batch["review"]]}

subset_with_keywords = subset.map(add_keywords, batched=True, batch_size=8)

5단계. 결과 확인

In [ ]:
# (단계 5) 결과 확인
pd.set_option("display.max_colwidth", 120)
df = subset_with_keywords.to_pandas()[["review", "keywords"]]


print(df.head(5))
